In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
covid_data_to_load = "owid-covid-data.csv"

# Read School and Student Data File and store into Pandas DataFrames
covid_data = pd.read_csv(covid_data_to_load)